# Visualize Database
- Following thread "http://www.austintaylor.io/d3/python/pandas/2016/02/01/create-d3-chart-python-force-directed/"
- Attempting to visualize the dvdrentals sample postgres database

## The Network Structure
- A dictionary with two lists, nodes and links.
- links contains the relationships between nodes
- nodes contains each individual node

```json
{
  "nodes":  [
    { "name": "desktop", "group":  1},
    { "name": "desktop/apples.txt", "group":  1},
    { "name": "desktop/pineapple/apples.txt", "group":  1},
    { "name": "desktop/bananas.txt", "group":  1}
  ],

  "links":  [
    { "source":  1,  "target":  0,  "value":  5555 },
    { "source":  2,  "target":  0,  "value":  1 },
    { "source":  3,  "target":  0,  "value": 1 }
  ]
}
```

## Setup

### Modules

In [1]:
import os
import pandas
import json

### Postgres SQL code generating the csv below

```SQL
SELECT 
    * 
INTO 
    new 
FROM
    (
        
        SELECT
            table_schema,
            table_name,
            column_name
        FROM
            information_schema.columns
        WHERE
            table_schema = 'public'
        
    ) 
    
    AS 
        x
```

### Data File Path 

In [2]:
data_path = "/users/danielcorcoran/desktop/github_repos/python_nb_networks/raw data/dvdrental_information_schema.csv"

### Read in data, add columns

In [3]:
raw_data = pandas.read_csv(data_path)

In [4]:
raw_data.head()

,table_schema,table_name,column_name
0,public,store,manager_staff_id
1,public,film,rating
2,public,film,special_features
3,public,inventory,last_update
4,public,staff,staff_id


## Manipulate raw data into desired format

In [5]:
subset1 = raw_data[["table_schema", "table_name"]]
subset2 = raw_data[["table_name", "column_name"]]

In [6]:
subset1.head()

,table_schema,table_name
0,public,store
1,public,film
2,public,film
3,public,inventory
4,public,staff


In [7]:
subset2.head()

,table_name,column_name
0,store,manager_staff_id
1,film,rating
2,film,special_features
3,inventory,last_update
4,staff,staff_id


In [8]:
subset1.columns = ["source", "target"]
subset2.columns = ["source", "target"]

In [9]:
subset1["structure"] = "table"
subset2["structure"] = "column"

In [10]:
data = pandas.concat([subset1, subset2], axis = 0)

In [11]:
data.head()

,source,target,structure
0,public,store,table
1,public,film,table
2,public,film,table
3,public,inventory,table
4,public,staff,table


In [12]:
data.shape

(260, 3)

In [13]:
data.drop_duplicates(inplace = True)

In [14]:
data.reset_index(drop = True, inplace = True)

## Build Json

### Set source and target column headers

In [15]:
source_header = "source"
target_header = "target"

### Convert items to strings

In [16]:
data[source_header] = data[source_header].astype(str)

In [17]:
data[target_header] = data[target_header].astype(str)

### Create list of unique targets to build nodes list

In [18]:
unique_nodes = sorted(list(data[target_header].unique()))

In [19]:
len(unique_nodes)

78

In [20]:
unique_sources = sorted(list(data[source_header].unique()))

In [21]:
for source in unique_sources:
    if source not in unique_nodes:
        unique_nodes.append(source)

### Build nodes list

In [22]:
#create nodes
nodes_list = []

In [23]:
source_list = list(data[source_header])
target_list = list(data[target_header])

for item in unique_nodes:
    
    #try get index within target series list first
    try:
        index = target_list.index(item)
        group_name = data.loc[index, "structure"]
        if group_name == "table":
            group_value = 0
        elif group_name == "column":
            group_value = 1
        else:
            group_value = -9990
    #else try get index within source list
    except:
        index = source_list.index(item)
        group_name = data.loc[index, "structure"]
        if group_name == "table":
            group_value = 0
        elif group_name == "column":
            group_value = 1
        else:
            group_value = -9990
    
    node_dictionary = {"name": item, "group": group_value}
    
    nodes_list.append(node_dictionary)

### Build links list

In [24]:
links_list = []
error_list = []

In [25]:
for index in range(data.shape[0]):
    
    target_value = data.loc[index, target_header]
    target_value_index = unique_nodes.index(target_value)  
    
    try:
        source_value = data.loc[index, source_header]
        source_value_index = unique_nodes.index(source_value)
        
        link_dictionary =  {"source":  source_value_index, 
                        "target":  target_value_index,
                        "value":  10 }
        links_list.append(link_dictionary)
        
    except:
        
        print(index, "HAD EN ERROR")
        error_list.append(index)

In [26]:
len(nodes_list)

79

### Create dictionary storing links_list and nodes_list together

In [27]:
json_data = {"links": links_list,"nodes":nodes_list}

## Export Json

### Convert python dictionary to json string

In [28]:
json_dump = json.dumps(json_data, indent=1, sort_keys=True)

### Export to filename 'pcap_export.json' to be used in index.html

In [29]:
json_out = open("json/dvdrental_database_network.json", "w")
json_out.write(json_dump)
json_out.close()